In [1]:
import cv2 
import csv
import mediapipe as mp 
import time 
import os 

In [7]:
mp_hands = mp.solutions.hands 
hands = mp_hands.Hands(max_num_hands=2, min_detection_confidence=0.6)
mp_drawing = mp.solutions.drawing_utils

output_dir = './data/dataset5'
os.makedirs(output_dir, exist_ok=True)


gesture_action = "EAT"

def record_hand_landmarks():
    cap = cv2.VideoCapture(0)
    recording = False
    landmarks_seq = []

    frame_rate = cap.get(cv2.CAP_PROP_FPS)
    frame_width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
    frame_height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        results = hands.process(frame_rgb)

        frame = cv2.cvtColor(frame_rgb, cv2.COLOR_RGB2BGR)
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
                if recording:
                    # extract the landmarks
                    landmarks = [lm for lm in hand_landmarks.landmark]
                    landmarks_flat = [coord for lm in landmarks for coord in (lm.x, lm.y, lm.z)]
                    landmarks_seq.append(landmarks_flat)  

        
        cv2.imshow("Hand Gesture REcording", frame)

        key = cv2.waitKey(5) & 0xFF
        if key == ord('r'):
            recording = True 
            landmarks_seq = []
            print("Recording started...")
        elif key == ord('s'):
            recording = False 
            print("Recording stopped.")
            if landmarks_seq:
                # save to CSV 
                gesture_name = gesture_action
                current_time = int(time.time())

                output_file = os.path.join(output_dir, f'{gesture_name}_{current_time}.csv')
                with open(output_file, 'w', newline='') as f:
                    writer = csv.writer(f)

                    # header 
                    header = ['frame'] + [f'{coord}_{i}' for i in range(21) for coord in ('x', 'y', 'z')] + ['frame_rate', 'frame_width', 'frame_height']
                    writer.writerow(header)

                    # write the data 
                    for i, frame_data in enumerate(landmarks_seq):
                        writer.writerow([i] + frame_data + [frame_rate, frame_width, frame_height])

                print(f'Saved {len(landmarks_seq)} frames to {output_file}')
        elif (key == 27) or (key == ord('q')):
            break 
    
    cap.release()
    cv2.destroyAllWindows()

record_hand_landmarks()
        






Recording started...
Recording stopped.
Saved 52 frames to ./data/dataset5\EAT_1719797140.csv
Recording started...
Recording stopped.
Saved 36 frames to ./data/dataset5\EAT_1719797144.csv
Recording started...
Recording stopped.
Saved 53 frames to ./data/dataset5\EAT_1719797150.csv
Recording started...
Recording stopped.
Saved 46 frames to ./data/dataset5\EAT_1719797158.csv
Recording started...
Recording stopped.
Saved 32 frames to ./data/dataset5\EAT_1719797163.csv
Recording started...
Recording stopped.
Saved 43 frames to ./data/dataset5\EAT_1719797187.csv
